In [1]:
import pandas as pd
import numpy as np
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Plotly Express


In [2]:
import plotly.express as px

In [3]:
df_s = pd.read_csv('南棟客梯.csv',index_col = 0)  
print(df_s.head())
print(df_s.tail())

  floor  hour direction     week    avgWait
0    B4     0        up  weekday  13.542857
1    B4     1        up  weekday   3.435000
2    B4     2        up  weekday  20.257500
3    B4     3        up  weekday  21.478000
4    B4     4        up  weekday   6.170000
     floor  hour direction     week    avgWait
1496     1    21        up  weekend  17.123382
1497     1    22      down  weekend  17.230000
1498     1    22        up  weekend  14.311524
1499     1    23      down  weekend  16.506154
1500     1    23        up  weekend  14.112249


plotly express 有點像是 ggplot in R，<br>
只要指定x,y,color等就能自動幫你分類，<br>
下面的圖表以南棟電梯的外部電梯呼叫資料為例，<br>
x軸為樓層、以方向分開並上色，最後以時間(小時)作為動畫影格。

In [4]:
floor_list = ['B4','B3', 'B2','B1','01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15']

fig = px.bar(data_frame=df_s, 
             
             ## 這裡的data frame都先用 R aggregate過了，以節省運算時間
             x = "floor", y = "avgWait",
             
             ## 以方向分兩張圖，並上色
             facet_col="direction",color="direction",
             
             ## 自動生成slider
             animation_frame="hour",
             
             labels={"avgWait":"sec",
                     "hour":"Local Time (GMT+8)"})

# X axis
fig.update_xaxes(nticks=len(floor_list), ##指定breaks的數量
                 
                 ## 手動將X的類別排序
                 type="category",
                 categoryarray=floor_list)

# Y axis
fig.update_yaxes(range=[0, 1000])

# Layout
fig.update_layout(title_text='Frequency of Elevator Calls by Floor')

fig.show()

In [5]:
fig = px.bar(data_frame=df_s, 
             x = "hour", y = "avgWait",
             facet_col="direction",color="direction",
             animation_frame="floor",
             labels={"avgWait":"sec",
                     "hour":"Local Time (GMT+8)"})
# X axis
fig.update_xaxes(nticks=len(floor_list),
                 type="category",
                 categoryarray=[0,24])

# Y axis
fig.update_yaxes(range=[0, 1000])

# Layout
fig.update_layout(title_text='Frequency of Elevator Calls by Time')

fig.show()

## Data

In [6]:
import plotly.graph_objects as go
from ipywidgets import widgets

將不同大樓資料混在一起，依大樓名稱貼標

In [7]:
## 檔案名(註:資料是arregate後的table)

files_list = ['研究大樓','北棟病床梯','南棟客梯']

df = pd.DataFrame()

for i in files_list:
    data = pd.read_csv(i+'.csv', index_col = 0)
    data['building'] = i
    print(i)
    print(data.shape)
    
    df = df.append(data, ignore_index = True) 

## 請檢查表格的數量是否正確 有些表格有缺失(e.g.北棟客梯)
    
print(df.shape)
print(df.head())

研究大樓
(1409, 6)
北棟病床梯
(1584, 6)
南棟客梯
(1501, 6)
(4494, 6)
  floor  hour direction     week   avgWait building
0     1     0      down  weekday   8.03000     研究大樓
1     1     0        up  weekday   3.78375     研究大樓
2     1     1      down  weekday   8.42500     研究大樓
3     1     1        up  weekday  11.13600     研究大樓
4     1     2      down  weekday   9.36500     研究大樓


## Dash

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash()
app.layout = html.Div([
    html.H1('Outer calls'),
    html.H3('By Floor'),
    dcc.Dropdown(
        id="building1",
        options=[
            {'label': '研究大樓', 'value': '研究大樓'},
            {'label': '北棟病床梯', 'value': '北棟病床梯'},
            {'label': '南棟電梯', 'value': '南棟電梯'}
        ],
        value='研究大樓'
    ),
    dcc.Dropdown(
        id="week1",
        options=[
            {'label': '全日', 'value': 'total'},
            {'label': '平日', 'value': 'weekday'},
            {'label': '假日', 'value': 'weekend'}
        ],
        value='total'
    ),
    dcc.Graph(id="graph1"),
    
    
    html.H3('By Hour'),
    dcc.Dropdown(
        id="building2",
        options=[
            {'label': '研究大樓', 'value': '研究大樓'},
            {'label': '北棟病床梯', 'value': '北棟病床梯'},
            {'label': '南棟電梯', 'value': '南棟電梯'}
        ],
        value='研究大樓'
    ),
        dcc.Dropdown(
        id="week2",
        options=[
            {'label': '全日', 'value': 'total'},
            {'label': '平日', 'value': 'weekday'},
            {'label': '假日', 'value': 'weekend'}
        ],
        value='total'
    ),
    dcc.Graph(id="graph2")
])

@app.callback(
    Output('graph1', 'figure'),
    [Input('building1', 'value'),
     Input('week1', 'value')])
def update_figure(selected_building, selected_week):
    
    ## Building
    temp = df[df.building == selected_building]
    
    ## Weekday / Weekend
    if (selected_week != 'total'):
        temp = temp[temp.week == selected_week]
        
        
    fig = px.bar(data_frame=temp,
                 x = "floor", y = "avgWait",
                 facet_col="direction",color="direction",
                 animation_frame="hour",
                 labels={"avgWait":"Average Waiting Time(sec)",
                         "hour":"Local Time (GMT+8)"})
    fig.update_xaxes(nticks=len(floor_list),
                     type="category",
                    categoryarray=floor_list)
    fig.update_yaxes(range=[0, 1000])
    fig.update_layout(title_text='Average Waiting Time of Elevator Calls in ' + selected_building + ' ('+ selected_week +')')
    
    return fig



@app.callback(
    Output('graph2', 'figure'),
    [Input('building2', 'value'),
     Input('week2', 'value')])
def update_figure(selected_building, selected_week):
    
    ## Building
    temp = df[df.building == selected_building]
    
    ## Weekday / Weekend
    if (selected_week != 'total'):
        temp = temp[temp.week == selected_week]
        
    
    fig = px.bar(data_frame=temp, 
             x = "hour", y = "avgWait",
             facet_col="direction",color="direction",
             animation_frame="floor",
             labels={"avgWait":"Average Waiting Time(sec)",
                     "hour":"Local Time (GMT+8)"})
    fig.update_xaxes(nticks=len(floor_list),
                     type="category",
                     categoryarray=[0,24])
    fig.update_yaxes(range=[0, 1000])
    fig.update_layout(title_text='Frequency of Elevator Calls by Time in ' + selected_building + ' ('+ selected_week +')')
    return fig
        
app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Debugger PIN: 345-200-069
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
